In [1]:
import qiskit

In [2]:
from qiskit_aer import Aer

In [3]:
from qiskit_aer import Aer

# Now you can use Aer.get_backend as intended
aer_simulator = Aer.get_backend('aer_simulator')
print(aer_simulator)

AerSimulator('aer_simulator')


In [5]:
# -*- coding: utf-8 -*-
"""
A mini-project simulating a simplified Quantum Key Distribution (QKD)
protocol with an eavesdropper (Eve).

This script demonstrates how Eve's measurement of a quantum state
can disturb the state, leading to a detectable difference in Bob's
measurements compared to what would be expected without eavesdropping.

Key Concepts:
- Quantum Circuit: Used to define quantum operations.
- Hadamard Gate (H): Creates superposition states (e.g., |+⟩ = (|0⟩ + |1⟩)/√2).
- Measurement: Collapses the quantum state to a classical outcome.
- Quantum Simulator: Used to run quantum circuits locally.
- Histogram: Visualizes the distribution of measurement outcomes.
"""

from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

def run_qkd_simulation(shots=1024):
    """
    Runs a simplified QKD simulation with an eavesdropper.

    Alice prepares a |+⟩ state.
    Eve intercepts, measures in the Hadamard basis, and resends.
    Bob receives and measures in the Hadamard basis.

    Args:
        shots (int): The number of times to run the circuit for measurement.

    Returns:
        tuple: A tuple containing Eve's measurement counts and Bob's measurement counts.
    """

    print(f"--- Starting QKD Simulation with {shots} shots ---")

    # 1. Alice prepares the QKD qubit in |+⟩ state
    # This circuit represents Alice's preparation of a single qubit
    # in the Hadamard basis (|+⟩ state).
    alice_circuit = QuantumCircuit(1, name="Alice_Prep")
    alice_circuit.h(0)  # Apply Hadamard gate to create |+⟩ = (|0⟩ + |1⟩)/√2

    print("\nAlice prepares a qubit in the |+⟩ state.")
    print(alice_circuit.draw(output='text', fold=-1))

    # 2. Eve intercepts and measures the qubit
    # Eve creates her own circuit to intercept Alice's qubit.
    # She applies a Hadamard gate (to measure in the Hadamard basis)
    # and then measures the qubit.
    eve_circuit = QuantumCircuit(1, 1, name="Eve_Intercept")
    eve_circuit.h(0)      # Eve measures in the Hadamard basis
    eve_circuit.measure(0, 0) # Eve measures the qubit

    print("\nEve intercepts the qubit, measures it in the Hadamard basis, and resends.")
    print(eve_circuit.draw(output='text', fold=-1))

    # Initialize the Aer simulator
    simulator = AerSimulator()

    # Simulate Eve's action
    # We transpile the circuit for the simulator to optimize it.
    transpiled_eve_circuit = transpile(eve_circuit, simulator)
    job_eve = simulator.run(transpiled_eve_circuit, shots=shots)
    eve_result = job_eve.result()
    eve_counts = eve_result.get_counts(transpiled_eve_circuit)

    print(f"\nEve's Measurement Results (counts): {eve_counts}")

    # 3. Bob receives Eve's resubmitted qubit and measures
    # Bob receives the qubit (which was potentially altered by Eve's measurement).
    # He also measures in the Hadamard basis.
    bob_circuit = QuantumCircuit(1, 1, name="Bob_Measurement")
    bob_circuit.h(0)      # Bob measures in the Hadamard basis
    bob_circuit.measure(0, 0) # Bob measures the qubit

    print("\nBob receives the (potentially altered) qubit and measures it in the Hadamard basis.")
    print(bob_circuit.draw(output='text', fold=-1))

    # Simulate Bob's action
    transpiled_bob_circuit = transpile(bob_circuit, simulator)
    job_bob = simulator.run(transpiled_bob_circuit, shots=shots)
    bob_result = job_bob.result()
    bob_counts = bob_result.get_counts(transpiled_bob_circuit)

    print(f"\nBob's Measurement Results (counts): {bob_counts}")

    print("\n--- Simulation Complete ---")

    return eve_counts, bob_counts

def visualize_results(eve_counts, bob_counts):
    """
    Visualizes the measurement results using a histogram.

    Args:
        eve_counts (dict): Dictionary of Eve's measurement counts.
        bob_counts (dict): Dictionary of Bob's measurement counts.
    """
    print("\n--- Plotting Results ---")
    fig = plot_histogram([eve_counts, bob_counts], legend=['Eve', 'Bob'], title="QKD Eavesdropping Simulation Results")
    plt.tight_layout() # Adjust layout to prevent labels from overlapping
    plt.show()
    print("--- Plot Displayed ---")

if __name__ == "__main__":
    # Run the simulation
    eve_measurements, bob_measurements = run_qkd_simulation(shots=2048) # Increased shots for better statistics

    # Visualize the outcomes
    visualize_results(eve_measurements, bob_measurements)

    print("\n--- End of Mini-Project ---")
    print("Notice that both Eve and Bob measure roughly 50/50 for '0' and '1'.")
    print("This is because Alice sent a |+⟩ state.")
    print("Eve measured it in the Hadamard basis, collapsing it to either |0⟩ or |1⟩.")
    print("When Bob then measured the (now |0⟩ or |1⟩) state in the Hadamard basis,")
    print("he also got a 50/50 chance of '0' or '1'.")
    print("In a real QKD protocol, this discrepancy (Alice sending |+⟩ and Bob not consistently receiving it)")
    print("would alert Alice and Bob to Eve's presence.")


--- Starting QKD Simulation with 2048 shots ---

Alice prepares a qubit in the |+⟩ state.
   ┌───┐
q: ┤ H ├
   └───┘

Eve intercepts the qubit, measures it in the Hadamard basis, and resends.
     ┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0 

Eve's Measurement Results (counts): {'0': 1006, '1': 1042}

Bob receives the (potentially altered) qubit and measures it in the Hadamard basis.
     ┌───┐┌─┐
  q: ┤ H ├┤M├
     └───┘└╥┘
c: 1/══════╩═
           0 

Bob's Measurement Results (counts): {'0': 1007, '1': 1041}

--- Simulation Complete ---

--- Plotting Results ---


<Figure size 640x480 with 0 Axes>

--- Plot Displayed ---

--- End of Mini-Project ---
Notice that both Eve and Bob measure roughly 50/50 for '0' and '1'.
This is because Alice sent a |+⟩ state.
Eve measured it in the Hadamard basis, collapsing it to either |0⟩ or |1⟩.
When Bob then measured the (now |0⟩ or |1⟩) state in the Hadamard basis,
he also got a 50/50 chance of '0' or '1'.
In a real QKD protocol, this discrepancy (Alice sending |+⟩ and Bob not consistently receiving it)
would alert Alice and Bob to Eve's presence.
